# 📄 テキストチャンク化教材：RAG のための事前処理と設計

この教材では、RAG（Retrieval-Augmented Generation）構成の事前処理として不可欠な「チャンク化（text chunking）」について、背景知識から Python での実装例までを段階的に学びます。Weaviate や LangChain などと組み合わせる前提の教材です。

## ✅ ステップ ①：チャンク化とは？

### 📌 概要

- チャンク（chunk）＝検索・埋め込み用の**適切な長さの分割テキスト**
- 多くの LLM はトークン数制限があるため、**長文は分割して扱う必要**がある
- 例：PDF・HTML・Markdown・議事録などの文書を 300〜500 文字ごとに分割

### 📌 なぜ重要か？

- 細かすぎる → コンテキストが失われる
- 長すぎる → 入力制限・ノイズが増える

## ✅ ステップ ②：分割の方法と戦略

### 📌 固定文字数で分割（シンプル）

In [ ]:
from textwrap import wrap

text = "...長いテキスト..."
chunks = wrap(text, width=300)
print(chunks)

### 📌 センテンス単位 + トークン考慮（NLTK など）

#### 英語など欧文

ライブラリで利用するNLTKデータを別途ダウンロードする  
辞書みたいなものなので結構容量があると思われるためセンテンス単位の分析をしたい人は各自で実施ください。


In [ ]:
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')

In [ ]:
import nltk
from nltk.tokenize import sent_tokenize

sentences = sent_tokenize(text)
chunks = []
buffer = ""
for sentence in sentences:
    if len(buffer + sentence) < 400:
        buffer += sentence + " "
    else:
        chunks.append(buffer.strip())
        buffer = sentence + " "
if buffer:
    chunks.append(buffer.strip())

#### 日本語（NLTK は不向き、代替として`janome`や`fugashi`などを使用）

In [ ]:
from janome.tokenizer import Tokenizer

text = "これは日本語の文です。文を分割して処理します。"
t = Tokenizer()
sentences = text.split("。")

chunks = []
buffer = ""
for sentence in sentences:
    sentence = sentence.strip()
    if not sentence:
        continue
    if len(buffer + sentence) < 400:
        buffer += sentence + "。"
    else:
        chunks.append(buffer.strip())
        buffer = sentence + "。"
if buffer:
    chunks.append(buffer.strip())

### 📌 LangChain の`RecursiveCharacterTextSplitter`

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
# Wikiより引用
# https://ja.wikipedia.org/wiki/%E7%94%9F%E6%88%90%E7%9A%84%E4%BA%BA%E5%B7%A5%E7%9F%A5%E8%83%BD
text = "生成的人工知能（せいせいてきじんこうちのう、英: generative artificial intelligence）または生成AI（せいせいエーアイ、英: GenAI）は、文字などの入力（プロンプト）に対してテキスト、画像、または他のメディアを応答として生成する人工知能システムの一種である[5][6]。ジェネレーティブAI、ジェネラティブAIともよばれる。生成的人工知能モデルは、訓練データの規則性や構造を訓練において学習することで、訓練データに含まれない新しいデータを生成することができる[7][8]。著名な生成AIシステムとして、OpenAIがGPT-3やGPT-4の大規模言語モデル[9]を使用して構築したチャットボットのChatGPT（および別形のBing Chat）や、GoogleがLaMDA基盤モデルに構築したチャットボットBardがある[10]。その他の生成AIモデルとして、Stable DiffusionやDALL-Eなどの人工知能アートシステムがあげられる[11]。生成AIは、アート、執筆、ソフトウェア開発、ヘルスケア、金融、ゲーム、マーケティング、ファッションなど、幅広い業界で応用できる可能性があるとされている[12][13]。生成AIへの投資は2020年代初頭に急増し、Microsoft、Google、Baiduなどの大企業だけでなく、多数の中小企業も生成AIモデルを開発している[5][14][15]。しかし、生成AIを訓練する目的での著作物の野放図な利用や人をだましたり操作したりするフェイクニュースやディープフェイクの作成など、生成AIの悪用の可能性も懸念されており[16][17][18]、欧州連合における人工知能法など法規制の議論も進んでいる[19][20]。また、効果的加速主義などの技術思想との関係も指摘されている[21]。"
splitter = RecursiveCharacterTextSplitter(
    chunk_size=100,
    chunk_overlap=20,
    separators=["\n\n", "\n", "。", "、", " "]
)
chunks = splitter.split_text(text)
for chunk in chunks:
    print(chunk)

## ✅ ステップ ③：正規表現を用いた前処理・整形

### 📌 ノイズ除去や整形の例

In [ ]:
import re

# 改行・空白の正規化
text = re.sub(r"\s+", " ", text)

# 特定の記号を除去
text = re.sub(r"[【】『』◇◆■□●○▲▼☆★▶︎→→]", "", text)

# セクション見出しで分割（例："第1章"）
chunks = re.split(r"第\d+章", text)

## ✅ ステップ ④：ファイルごとのチャンク化

### 📌 PDF の場合（PyMuPDF）

In [ ]:
import fitz  # PyMuPDF

doc = fitz.open("document.pdf")
text = "\n".join([page.get_text() for page in doc])

### 📌 Markdown/HTML/Text

In [ ]:
with open("file.md", encoding="utf-8") as f:
    text = f.read()

## ✅ ステップ ⑤：チャンクにメタデータを付加

### 📌 ファイル名・ページ番号などを保持

In [ ]:
chunk_records = []
for i, chunk in enumerate(chunks):
    chunk_records.append({
        "content": chunk,
        "source": "document.pdf",
        "page": i + 1
    })

## ✅ ステップ ⑥：ベクトルデータベース(Weaviate) への登録

### 📌 1.コレクションを作成

In [ ]:
import weaviate
import weaviate.classes.config as wvcc
from dotenv import load_dotenv

import os
load_dotenv()

# Weaviateにローカル接続
client = weaviate.connect_to_local()

# スキーマが未作成の場合は事前に作成する
if not client.collections.exists("DocumentChunk"):
    collection = client.collections.create(
        name="DocumentChunk",
        description="チャンクしたデータを保存する演習用コレクション",
        properties=[
            wvcc.Property(name="content", data_type=wvcc.DataType.TEXT),
            wvcc.Property(name="source", data_type=wvcc.DataType.TEXT),
            wvcc.Property(name="page", data_type=wvcc.DataType.INT)
        ],
        vectorizer_config=wvcc.Configure.Vectorizer.text2vec_azure_openai(
            base_url=os.environ.get("AZURE_OPENAI_ENDPOINT"),
            deployment_id=os.environ.get(
                "AZURE_OPENAI_EMBEDDING_DEPLOYMENT_MODEL"),
            resource_name=os.environ.get("AZURE_OPENAI_RESOURCE_NAME")
        )
    )
else:
    print(f"すでにコレクションが存在します: DocumentChunk")

client.close()

### 📌 2.チャンクを登録

In [ ]:
import weaviate
from dotenv import load_dotenv

load_dotenv()

# Weaviateにローカル接続
client = weaviate.connect_to_local()
doc_collection = client.collections.get("DocumentChunk")

# チャンクを登録
for i, record in enumerate(chunk_records):
    # サンプルドキュメントを登録
    # recordにはDocumentChunkで定義された書式のデータ
    # 例) {"content": "XXXX", "source":"YYYY", "page": 1}
    uuid = doc_collection.data.insert(properties=record)
    print(f"Inserted object no:{i}、UUID: {uuid}")

client.close()

## ✅ 実践課題

### 📁 課題 1：任意のテキストファイルを 300 文字ごとに分割し、チャンク一覧を出力

### 📁 課題 2：Markdown 文書から文単位チャンクを抽出し、章ごとのチャンク一覧を作成

### 📁 課題 3：チャンクごとにメタデータ（セクション名・出典など）を保持し、CSV または JSON に出力

### 🚀 最終ゴール

- LLM やベクトル DB と組み合わせる前提として、**扱いやすく検索に適したテキストチャンクを自動生成**できるようになること。